In [1]:
from konlpy.tag import Mecab
from MeCab import Tagger
MeCab = Mecab(dicpath="/usr/local/lib/mecab/dic/mecab-ko-dic")

In [2]:
def before(text):
    
    # mecabsplit 함수는 mecab_tagger, text, pos를 매개변수로 사용함.
    def mecabsplit(mecab_tagger,text, pos): 

        r=[] # r 이라는 리스트를 생성
        inputs = mecab_tagger.parse(text)

        t = inputs.split('\n')[:-2]

        for i in t:
            field = i.split('\t')
            if field[1].split(',')[-1] is not '*':
                r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
            else:
                r.append( (field[0],field[1].split(',')[0]) )
        if pos:
            return r
        else:
            return [ x[0] for x in r ]
    
    mecab_tagger = Tagger() # Mecab.Tagger()인식을 못해서 from Mecab import Tagger 선언 후 Tagger로 변경함. 2022.07.27
    
    a = mecab_tagger.parse(text)
    t = a.split('\n')[:-2]
    tokenpointer = []
    pointeroffset = 0

    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1] is not '*':
            currentptr = text.index(field[0], pointeroffset)
            for x in field[1].split(',')[-1].split('+'):
                try:
                    w = x.split('/')[0]
                    temp = field[0].index(w)
                    tokenpointer.append((currentptr + temp, currentptr + temp +len(w)))
                except:
                    tokenpointer.append((currentptr, currentptr + len(field[0])))
            pointeroffset = currentptr + len(field[0])
        else:
            currentptr = text.index(field[0], pointeroffset)
            tokenpointer.append( (currentptr, currentptr + len(field[0])) )
            pointeroffset = currentptr + len(field[0])
    words = mecabsplit(mecab_tagger, text, False)
    tokens = []
    offset = 0
    # for word in words:
    #     word_offset = tokenpointer[words.index(word, offset)][0]
    #     tokens.append(Token(word, word_offset))
    #     offset +=1      

    return words  


def after(text):
    mecab_tagger = Tagger() # Mecab.Tagger()인식을 못해서 from Mecab import Tagger 선언 후 Tagger로 변경함. 2022.07.27

    a = mecab_tagger.parse(text)
    t = a.split('\n')[:-2]
    words = []
    tokenpointer = []
    pointeroffset = 0

    for i in t:
        field = i.split('\t')
        words.append(field[0])
        currentptr = text.index(field[0], pointeroffset)
        tokenpointer.append( (currentptr, currentptr + len(field[0])) )
        pointeroffset = currentptr + len(field[0])

    # tokens = []
    # for word,offset in zip(words,tokenpointer):
    #     tokens.append(Token(word,offset[0]))
    
    return words

In [16]:
text = "따뜻한 아메리카노 한잔 주세요"

In [17]:
before(text)

['따뜻', '하', 'ᆫ', '아메리카노', '한', '잔', '주세', '이', '오']

In [18]:
after(text)

['따뜻', '한', '아메리카노', '한잔', '주세', '요']

In [7]:
MeCab.pos("아메리카노 뜨겁게 해서 한잔 주세요")

[('아메리카노', 'NNP'),
 ('뜨겁', 'VA'),
 ('게', 'EC'),
 ('해서', 'VV+EC'),
 ('한잔', 'NNG'),
 ('주세', 'NNG'),
 ('요', 'VCP+EC')]